In [1]:
from datasets import load_from_disk
import re
from typing import List, Tuple

dataset = load_from_disk("../generated_data/TUNING_final_summary")
train_dataset = dataset['train']
dev_dataset = dataset['dev']
test_dataset = dataset['test']
print('selesai memuat dataset...')

c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


selesai memuat dataset...


In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['query', 'passages', 'summary', 'final_summary', 'answer', 'generated_results'],
        num_rows: 4542
    })
    dev: Dataset({
        features: ['query', 'passages', 'summary', 'final_summary', 'answer', 'generated_results'],
        num_rows: 1143
    })
    test: Dataset({
        features: ['query', 'passages', 'summary', 'final_summary', 'answer', 'generated_results'],
        num_rows: 565
    })
})

In [ ]:


def split_each_passages(example):
    # Pisahkan teks menjadi pasangan Judul-Teks berdasarkan '\n\n'
    passages = example["passages"].split('\n\n')
    
    # List untuk menyimpan hasil dari prepare_context_chunks untuk setiap bagian
    all_chunks = []
    
    # Proses setiap pasangan Judul-Teks
    for passage in passages:
        # Panggil fungsi prepare_context_chunks pada setiap bagian
        all_chunks.extend(prepare_context_chunks(passage))
    
    # Kembalikan hasil ke dalam format baru
    return {"context_chunks": all_chunks}

def split_between_title_and_text(text: str) -> Tuple[str, str]:
    """Memisahkan bagian Judul dan Teks dari input lengkap."""
    title, content = text.split("Teks:", 1)
    return title.strip(), content.strip()

def split_sentences(text: str) -> List[str]:
    """Membagi teks panjang menjadi kalimat-kalimat."""
    text = re.sub(r'(\.\s?)\[\d+\]', r'\1', text)

    matches = list(re.finditer(r'\.(?=\s+[A-Z])', text))
    sentences = []
    start = 0

    for match in matches:
        end = match.end()
        candidate = text[start:end].strip()

        before_dot = text[start:match.start()].strip()
        if re.search(r'\b([A-Z][a-z]?|[A-Z](?:\.[A-Z])+)$', before_dot):
            continue

        sentences.append(text[start:end].strip())
        start = end

    if start < len(text):
        sentences.append(text[start:].strip())

    return sentences

def create_rolling_segments(title: str, sentences: List[str], window_size: int = 3) -> List[str]:
    """Membuat rolling window dari kalimat-kalimat dan menyisipkan judul di awal setiap segmen."""
    segments = []
    if len(sentences) < window_size:
        segments.append(f"{title}\n{' '.join(sentences)}")
    else:
        for i in range(len(sentences) - window_size + 1):
            group = sentences[i:i + window_size]
            combined = title + "\n" + " ".join(group)
            segments.append(combined)
    return segments

def prepare_context_chunks(text: str) -> List[str]:
    """Fungsi utama: memproses teks penuh menjadi rolling segments."""
    judul, teks = split_between_title_and_text(text)
    kalimat_list = split_sentences(teks)
    segments = create_rolling_segments(judul, kalimat_list, window_size=3)
    return segments



# Terapkan fungsi ke kolom "passages" dan simpan hasilnya di kolom baru
train_dataset = train_dataset.map(split_each_passages)
dev_dataset = dev_dataset.map(split_each_passages)
test_dataset = test_dataset.map(split_each_passages)
print('selesai membuat chunks...')

c:\Users\LENOVO\anaconda3\envs\recomp\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [16]:
import torch
import torch.nn.functional as F
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm

# Load model dan tokenizer
tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-small')
model = AutoModel.from_pretrained('intfloat/multilingual-e5-small')
model.eval()

# Fungsi pooling dari model card
def average_pool(last_hidden_states, attention_mask):
    last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
    return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

# Fungsi pencocokan top-3
def get_top3_chunks(query, chunks):
    if not chunks:
        return [], [], []

    input_texts = ["query: " + query] + ["passage: " + chunk for chunk in chunks]
    batch = tokenizer(input_texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

    with torch.no_grad():
        outputs = model(**batch)
    embeddings = average_pool(outputs.last_hidden_state, batch['attention_mask'])
    embeddings = F.normalize(embeddings, p=2, dim=1)

    query_embedding = embeddings[0]
    chunk_embeddings = embeddings[1:]
    scores = (query_embedding @ chunk_embeddings.T) * 100

    topk = torch.topk(scores, k=min(3, len(chunks)))  # Antisipasi jika chunk < 3
    top_chunks = [chunks[i] for i in topk.indices.tolist()]
    top_scores = topk.values.tolist()
    top_indices = topk.indices.tolist()

    return top_chunks, top_scores, top_indices

# Terapkan ke dataset
def process_dataset(dataset):
    top_chunks_all = []
    top_scores_all = []
    top_indices_all = []

    for example in tqdm(dataset, desc="Processing"):
        query = example['query']
        chunks = example['context_chunks']

        top_chunks, top_scores, top_indices = get_top3_chunks(query, chunks)

        top_chunks_all.append(top_chunks)
        top_scores_all.append(top_scores)
        top_indices_all.append(top_indices)

    # Tambahkan kolom baru
    dataset = dataset.add_column("top_chunks", top_chunks_all)
    dataset = dataset.add_column("top_chunk_scores", top_scores_all)
    dataset = dataset.add_column("top_chunk_indices", top_indices_all)
    return dataset

# Proses semua split
train_dataset = process_dataset(train_dataset)
dev_dataset = process_dataset(dev_dataset)
test_dataset = process_dataset(test_dataset)


Processing: 100%|██████████| 565/565 [01:52<00:00,  5.00it/s]


In [17]:
from datasets import DatasetDict

dataset = DatasetDict({
    "train": train_dataset,
    "dev": dev_dataset,
    "test": test_dataset
})
dataset.save_to_disk("../generated_data/top_reranked_chunks")


Saving the dataset (1/1 shards): 100%|██████████| 565/565 [00:00<00:00, 38521.76 examples/s]


In [ ]:
# full_text = '''Judul: Ernest Douwes Dekker 
# Teks: Ernest Douwes Dekker wafat dini hari tanggal 28 Agustus 1950 (tertulis di batu nisannya; 29 Agustus 1950 versi van der Veur, 2006) dan dimakamkan di TMP Cikutra, Bandung.'''

# # full_text = '''Judul: Kabupaten Probolinggo
# # Teks: Bintang berwarna kuning melambangkan ketuhanan yang maha esa. Sungai sebagai tanda "sungai banger". Yang menjadi asal nama kabupaten ini. Angin berwarna merah dan putih sebagai ciri khas Kabupaten Probolinggo yang terkenal dengan sebutan "Angin Gending". Angin ini biasanya datang pada musim kemarau dari arah tenggara Gunung sebagai tanda Gunung Bromo yang terletak di Pegunungan Tengger. Dataran tanah berwarna hijau, merupakan pertanda daerah daratan Kab.Probolinggo yang cukup subur. Gelombang air laut, yang menunjukkan letak Kabupaten Probolinggo di tepi Selat Madura Daun anggur sebanyak 4 buah berwarna hijau muda, dengan 17 buah anggur. Menunjukkan hasil buah-buahan khas Probolinggo. Daun mangga sebanyak lima, dan buah mangga sebanyak 8 buah.Menunjukkan hasil buah-buahan khas Probolinggo. Susunan buah anggur-buah mangga-daun anggur-daun mangga melambangkan tanggal kemerdekaan Indonesia 17-8-45 (17 Agustus 1945) Pita dasar dengan warna putih bertuliskan Prasadja Ngesti Wibawa yang beartikan "Bersahaja menciptakan kemuliaan" Warna kuning berartikan keagungan, keluhuran, dan kemuliaan. Warna biru berartikan kesetiaan. Warna hijau berartikan kesuburan, kemakmuran.'''
# hasil = prepare_context_chunks(full_text)

# # Tampilkan hasil
# for i, segmen in enumerate(hasil, 1):
#     print(f"[Segment {i}]\n{segmen}\n")

Map: 100%|██████████| 565/565 [00:00<00:00, 3395.50 examples/s]
